<a href="https://colab.research.google.com/github/Zhaotai924/Pytorch-Tutorial-Youtube/blob/main/cnn_color__version2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input, UpSampling2D, LeakyReLU, BatchNormalization, Activation, Add, Input, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import HeNormal
from skimage.color import lab2rgb, rgb2lab, rgb2gray
import os
import glob
import cv2

In [ ]:
# 定义数据集路径
train_data_path = '/content/drive/MyDrive/train/images'

In [ ]:
class ImageColorizationDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_dir, image_size=(224, 224), batch_size=16):
        self.image_dir = image_dir
        self.image_size = image_size
        self.batch_size = batch_size
        self.image_files = glob.glob(os.path.join(image_dir, '*.jpg'))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.image_files) / self.batch_size))

    def __getitem__(self, index):
        batch_files = self.image_files[index * self.batch_size:(index + 1) * self.batch_size]
        grayscale_images, color_images = self.__data_generation(batch_files)
        return grayscale_images, color_images

    def on_epoch_end(self):
        np.random.shuffle(self.image_files)

    def __data_generation(self, batch_files):
        grayscale_images = []
        color_images = []

        for file_name in batch_files:
            img = cv2.imread(file_name)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, self.image_size)

            lab_img = rgb2lab(img)
            l = lab_img[:, :, 0]
            ab = lab_img[:, :, 1:] / 128.0  # normalize A and B channels to [-1, 1]

            l = l[..., np.newaxis] / 100.0  # normalize L channel to [0, 1]

            grayscale_images.append(l)
            color_images.append(ab)

        grayscale_images = np.array(grayscale_images)
        color_images = np.array(color_images)

        return grayscale_images, color_images

In [ ]:
train_data_generator = ImageColorizationDataGenerator(train_data_path, batch_size=16)

第一版优化

In [ ]:
def residual_block(x, filters, kernel_size=(3, 3), stride=1):
    y = Conv2D(filters, kernel_size, strides=stride, padding='same')(x)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = Conv2D(filters, kernel_size, strides=stride, padding='same')(y)
    y = BatchNormalization()(y)

    if x.shape[-1] != filters:
        x = Conv2D(filters, (1, 1), strides=stride, padding='same')(x)

    out = Add()([x, y])
    out = Activation('relu')(out)
    return out

In [ ]:
def create_colorization_cnn(input_tensor):
    """构建优化的卷积神经网络部分"""
    x = Conv2D(32, (3, 3), padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)

    x = residual_block(x, 64)
    x = MaxPooling2D((2, 2), padding='same')(x)

    x = residual_block(x, 128)
    x = MaxPooling2D((2, 2), padding='same')(x)

    x = residual_block(x, 128)

    x = UpSampling2D((2, 2))(x)
    x = residual_block(x, 64)

    x = UpSampling2D((2, 2))(x)
    x = residual_block(x, 32)

    x = UpSampling2D((2, 2))(x)
    output_tensor = Conv2D(2, (3, 3), activation='tanh', padding='same')(x)

    return output_tensor


原版

In [ ]:
# def create_colorization_cnn(input_tensor):
#     """构建卷积神经网络部分"""
#     x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_tensor)
#     x = MaxPooling2D((2, 2), padding='same')(x)
#     x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
#     x = MaxPooling2D((2, 2), padding='same')(x)
#     x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
#     x = MaxPooling2D((2, 2), padding='same')(x)

#     x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
#     x = UpSampling2D((2, 2))(x)
#     x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
#     x = UpSampling2D((2, 2))(x)
#     x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
#     x = UpSampling2D((2, 2))(x)

#     output_tensor = Conv2D(2, (3, 3), activation='tanh', padding='same')(x)
#     return output_tensor

In [ ]:
# 定义输入层，输入形状为224x224x1（灰度图）
input_tensor = Input(shape=(224, 224, 1))

# 调用函数构建卷积神经网络部分
output_tensor = create_colorization_cnn(input_tensor)

# 构建模型
model = Model(inputs=input_tensor, outputs=output_tensor)

# 编译模型
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mse', metrics=['accuracy'])

# 训练模型
model.fit(train_data_generator, epochs=20)

Epoch 1/20
353/353 [==============================] - 128s 343ms/step - loss: 0.0167 - accuracy: 0.6472
Epoch 2/20
353/353 [==============================] - 70s 199ms/step - loss: 0.0011 - accuracy: 0.6779
Epoch 3/20
353/353 [==============================] - 70s 197ms/step - loss: 7.7082e-04 - accuracy: 0.7499
Epoch 4/20
353/353 [==============================] - 71s 201ms/step - loss: 6.3011e-04 - accuracy: 0.7988
Epoch 5/20
353/353 [==============================] - 70s 199ms/step - loss: 5.5349e-04 - accuracy: 0.8088
Epoch 6/20
353/353 [==============================] - 69s 196ms/step - loss: 5.0581e-04 - accuracy: 0.8147
Epoch 7/20
353/353 [==============================] - 69s 196ms/step - loss: 4.7672e-04 - accuracy: 0.8188
Epoch 8/20
353/353 [==============================] - 70s 198ms/step - loss: 4.4341e-04 - accuracy: 0.8227
Epoch 9/20
353/353 [==============================] - 70s 198ms/step - loss: 4.2789e-04 - accuracy: 0.8253
Epoch 10/20
353/353 [=======================

In [ ]:
import os
import glob
import numpy as np
import cv2
from keras.models import load_model
from skimage.color import lab2rgb

# 定义测试图像路径
test_data_path = "/content/drive/MyDrive/test_color/images"

# 保存预测结果的路径
save_path = "/content/drive/MyDrive/content/sample_data/"

# 加载测试图像文件列表
test_image_files = glob.glob(os.path.join(test_data_path, '*.jpg'))

# 遍历测试图像并进行预测
for file_name in test_image_files:
    # 读取测试图像
    img = cv2.imread(file_name)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 将图像转换为灰度图并调整大小
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_gray = cv2.resize(img_gray, (224, 224))
    img_gray = img_gray[..., np.newaxis] / 255.0  # 归一化到 [0, 1] 范围内

    # 扩展维度以匹配模型输入
    img_gray_input = np.expand_dims(img_gray, axis=0)

    # 使用模型进行预测
    predicted_ab = model.predict(img_gray_input)

    # 反归一化，恢复原始尺度的AB通道
    predicted_ab = predicted_ab * 128.0

    # 将灰度图与预测的AB通道合并成LAB图像
    lab_image = np.zeros((224, 224, 3))
    lab_image[:,:,0] = img_gray[:,:,0] * 100.0  # L通道
    lab_image[:,:,1:] = predicted_ab

    # 转换为RGB图像
    predicted_rgb = lab2rgb(lab_image)

    # 保存预测结果
    save_name = os.path.join(save_path, os.path.basename(file_name))
    cv2.imwrite(save_name, cv2.cvtColor((predicted_rgb * 255).astype(np.uint8), cv2.COLOR_RGB2BGR))


1/1 [==============================] - 0s 22ms/step


In [ ]:
import os
from PIL import Image
import numpy as np

# 文件夹路径，包含多个JPEG图像
folder_path = '/content/drive/MyDrive/content/sample_data'

# 获取文件夹中的所有JPEG文件
image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]

# 读取所有JPEG图像并转换为NumPy数组
image_arrays = []
for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    image = Image.open(image_path)
    image_array = np.array(image)
    image_arrays.append(image_array)

# 将所有图像数组堆叠到一个数组中
all_images_array = np.stack(image_arrays)

# 保存NumPy数组为npy文件
npy_path = 'images.npy'
np.save(npy_path, all_images_array)

print(f"All images saved as NumPy array at {npy_path}")


All images saved as NumPy array at images.npy
